In [1]:
import arcpy
import pandas as pd
import os

### Import the csv file and convert it to XY table to point

In [2]:
arcpy.management.XYTableToPoint(
    in_table="combined_data_0930.csv",
    out_feature_class=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project GIS5572\Final project GIS5572.gdb\combined_data_0930_XYTableToPoint",
    x_field="X",
    y_field="Y",
    z_field="Cumulative_GDD",
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],VERTCS["WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
)

<Result 'C:\\Users\\samik\\OneDrive\\Documents\\ArcGIS\\Projects\\Final project GIS5572\\Final project GIS5572.gdb\\combined_data_0930_XYTableToPoint'>

In [3]:
### Interpolation of those datasets

In [4]:
# IDW
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project GIS5572\Final project GIS5572.gdb"):
    out_raster = arcpy.sa.Idw(
        in_point_features="combined_data_0930_XYTableToPoint",
        z_field="Cumulative_GDD",
        cell_size=0.0218500823959998,
        power=2,
        search_radius="VARIABLE 12",
        in_barrier_polyline_features=None
    )
    out_raster.save(r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project GIS5572\Final project GIS5572.gdb\Idw_combined1")

In [5]:
#Krigging Ordinary
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project GIS5572\Final project GIS5572.gdb"):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="combined_data_0930_XYTableToPoint",
        z_field="Cumulative_GDD",
        kriging_model="Spherical 0.021850 # # #",
        cell_size=0.0218500823959998,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project GIS5572\Final project GIS5572.gdb\Kriging_comb1")

In [6]:
#Krigging Universal
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project GIS5572\Final project GIS5572.gdb"):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="combined_data_0930_XYTableToPoint",
        z_field="Cumulative_GDD",
        kriging_model="LinearDrift 0.021850 # # #",
        cell_size=0.0218500823959998,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project GIS5572\Final project GIS5572.gdb\Kriging_comb2")

### Raster to point

In [7]:
arcpy.conversion.RasterToPoint(
    in_raster="Idw_combined1",
    out_point_features=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project GIS5572\Final project GIS5572.gdb\RasterT_Idw_com1",
    raster_field="Value"
)
arcpy.conversion.RasterToPoint(
    in_raster="Kriging_comb1",
    out_point_features=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project GIS5572\Final project GIS5572.gdb\RasterT_Kriging1",
    raster_field="Value"
)
arcpy.conversion.RasterToPoint(
    in_raster="Kriging_comb2",
    out_point_features=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project GIS5572\Final project GIS5572.gdb\RasterT_Kriging2",
    raster_field="Value"
)

<Result 'C:\\Users\\samik\\OneDrive\\Documents\\ArcGIS\\Projects\\Final project GIS5572\\Final project GIS5572.gdb\\RasterT_Kriging2'>

### Exploratory analysis

In [8]:

arcpy.ga.ExploratoryInterpolation(
    in_features="RasterT_Idw_com1",
    value_field="pointid",
    out_cv_table=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project GIS5572\Final project GIS5572.gdb\ExploratoryInterpolation2",
    out_geostat_layer=None,
    interp_methods="SIMPLE_KRIGING;ORDINARY_KRIGING;UNIVERSAL_KRIGING",
    comparison_method="SINGLE",
    criterion="ACCURACY",
    criteria_hierarchy="ACCURACY PERCENT #",
    weighted_criteria="ACCURACY 1",
    exclusion_criteria=None
)

<Result 'C:\\Users\\samik\\OneDrive\\Documents\\ArcGIS\\Projects\\Final project GIS5572\\Final project GIS5572.gdb\\ExploratoryInterpolation2'>